# Donkey Colab with GPU

Donkey Colab with GPU.

# TensorFlowをVersion Down

Colab環境では、TensorFlowが1.13.0になっているので、1.12.0にDown gradeします。

In [0]:
!pip install tensorflow==1.12.0
!pip install q keras==2.2.4

# DonkeyレポジトリをGit clone

In [0]:
!git clone -b dev https://github.com/autorope/donkeycar.git donkey

Cloning into 'donkey'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 6688 (delta 0), reused 0 (delta 0), pack-reused 6687
Receiving objects: 100% (6688/6688), 41.53 MiB | 9.59 MiB/s, done.
Resolving deltas: 100% (3851/3851), done.


# インストール

In [0]:
!pip3 install -e donkey

Obtaining file:///content/donkey
    100% |████████████████████████████████| 51kB 2.1MB/s 
    100% |████████████████████████████████| 225kB 7.5MB/s 
    100% |████████████████████████████████| 122kB 35.0MB/s 
    100% |████████████████████████████████| 194kB 35.2MB/s 
  Running setup.py develop for donkeycar


# Projectの作成

In [0]:
!donkey createcar --template donkey2  /content/mycar

using donkey version: 2.5.7 ...
Creating car folder: /content/mycar
making dir  /content/mycar
Creating data & model folders.
making dir  /content/mycar/models
making dir  /content/mycar/data
making dir  /content/mycar/logs
Copying car application template: donkey2
Copying car config defaults. Adjust these before starting your car.
Donkey setup complete.


# datas.zipのアップロード

**ファイル-アップロード**を選択し、PCからdatas.zipをColabにアップロードします。

In [0]:
!mv /content/datas.zip /content/mycar/data/

In [0]:
%cd /content/mycar/data/

!unzip datas.zip

/content/mycar/data
Archive:  datas.zip
   creating: tub_01_19-01-26/
  inflating: tub_01_19-01-26/record_2918.json  
  inflating: tub_01_19-01-26/record_2464.json  
  inflating: tub_01_19-01-26/1451_cam-image_array_.jpg  
  inflating: tub_01_19-01-26/record_4369.json  
  inflating: tub_01_19-01-26/3677_cam-image_array_.jpg  
  inflating: tub_01_19-01-26/record_2718.json  
  inflating: tub_01_19-01-26/record_4078.json  
  inflating: tub_01_19-01-26/record_1017.json  
  inflating: tub_01_19-01-26/record_1909.json  
  inflating: tub_01_19-01-26/4577_cam-image_array_.jpg  
  inflating: tub_01_19-01-26/record_3844.json  
  inflating: tub_01_19-01-26/277_cam-image_array_.jpg  
  inflating: tub_01_19-01-26/record_2516.json  
  inflating: tub_01_19-01-26/record_3162.json  
  inflating: tub_01_19-01-26/1445_cam-image_array_.jpg  
  inflating: tub_01_19-01-26/4233_cam-image_array_.jpg  
  inflating: tub_01_19-01-26/record_204.json  
  inflating: tub_01_19-01-26/record_2638.json  
  inflating: t

# 学習

学習する前に、ここまでインストールしたライブラリを有効にするために
**ランタイム-ランタイムの再起動**
を実行します。

In [0]:
#!/usr/bin/env python3

import os

from docopt import docopt
import donkeycar as dk
import tensorflow as tf
from donkeycar.parts.camera import PiCamera
from donkeycar.parts.transform import Lambda
from donkeycar.parts.keras import KerasLinear
from donkeycar.parts.actuator import PCA9685, PWMSteering, PWMThrottle
from donkeycar.parts.datastore import TubGroup, TubWriter
from donkeycar.parts.web_controller import LocalWebController
from donkeycar.parts.clock import Timestamp
from donkeycar.parts.datastore import TubGroup, TubWriter
from donkeycar.parts.keras import KerasLinear
from donkeycar.parts.transform import Lambda


def train(cfg, tub_names, new_model_path, base_model_path=None):
    """
    use the specified data in tub_names to train an artifical neural network
    saves the output trained model as model_name
    """
    X_keys = ['cam/image_array']
    y_keys = ['user/angle', 'user/throttle']

    new_model_path = os.path.expanduser(new_model_path)

    kl = KerasLinear()
    if base_model_path is not None:
        base_model_path = os.path.expanduser(base_model_path)
        kl.load(base_model_path)

    print('tub_names', tub_names)
    if not tub_names:
        tub_names = os.path.join("/content/mycar/data/", '*')
    tubgroup = TubGroup(tub_names)
    train_gen, val_gen = tubgroup.get_train_val_gen(X_keys, y_keys,
                                                    batch_size=128,
                                                    train_frac=0.8)

    total_records = len(tubgroup.df)
    total_train = int(total_records * 0.8)
    total_val = total_records - total_train
    print('train: %d, validation: %d' % (total_train, total_val))
    steps_per_epoch = total_train //128
    print('steps_per_epoch', steps_per_epoch)

    kl.train(train_gen,
             val_gen,
             saved_model_path=new_model_path,
             steps=steps_per_epoch,
             train_split=0.8)


if __name__ == '__main__':
    with tf.device('/gpu:0'):
      cfg = None

      tub = None
      new_model_path = "/content/mycar/models/mypilot"
      base_model_path = None
      cache = None
      train(cfg, tub, new_model_path, base_model_path)

using donkey version: 2.5.7 ...
tub_names None
train: 3688, validation: 922
steps_per_epoch 28
Epoch 1/100
27/28 [===========================>..] - ETA: 0s - loss: 25.4787 - angle_out_loss: 23.5847 - throttle_out_loss: 27.3727
Epoch 00001: val_loss improved from inf to 0.19389, saving model to /content/mycar/models/mypilot
28/28 [==============================] - 13s 447ms/step - loss: 24.5767 - angle_out_loss: 22.7560 - throttle_out_loss: 26.3975 - val_loss: 0.1939 - val_angle_out_loss: 0.3473 - val_throttle_out_loss: 0.0405
Epoch 2/100
27/28 [===========================>..] - ETA: 0s - loss: 0.1859 - angle_out_loss: 0.3239 - throttle_out_loss: 0.0479
Epoch 00002: val_loss improved from 0.19389 to 0.17783, saving model to /content/mycar/models/mypilot
28/28 [==============================] - 7s 254ms/step - loss: 0.1856 - angle_out_loss: 0.3231 - throttle_out_loss: 0.0480 - val_loss: 0.1778 - val_angle_out_loss: 0.3242 - val_throttle_out_loss: 0.0314
Epoch 3/100
27/28 [===============